## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-30-21-47-19 +0000,nypost,"Islamic Republic close to collapse in Iran, ci...",https://nypost.com/2025/08/30/world-news/islam...
1,2025-08-30-21-42-48 +0000,nypost,NYC baby clinging to life after being repeated...,https://nypost.com/2025/08/30/us-news/nyc-baby...
2,2025-08-30-21-23-32 +0000,startribune,He couldn’t run. So they covered him instead.,https://www.startribune.com/he-couldnt-run-so-...
3,2025-08-30-21-10-12 +0000,nyt,"Xi Uses Summit, Parade and History to Flaunt C...",https://www.nytimes.com/2025/08/30/world/asia/...
4,2025-08-30-21-07-00 +0000,nyt,Israel Military Says It Recovered Body of Two ...,https://www.nytimes.com/2025/08/29/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
183,trump,35
210,court,12
277,tariffs,10
64,israeli,9
63,killed,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
145,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...,127
175,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...,100
199,2025-08-29-22-04-53 +0000,nypost,Trump’s tariffs ruled illegal in stunning blow...,https://nypost.com/2025/08/29/us-news/trumps-t...,92
196,2025-08-29-22-22-47 +0000,cbc,U.S. appeals court finds many Trump tariffs un...,https://www.cbc.ca/news/world/appeals-court-tr...,87
29,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...,76


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
145,127,2025-08-30-04-15-00 +0000,wsj,"A federal judge in Washington, D.C., temporari...",https://www.wsj.com/politics/policy/judge-bloc...
27,44,2025-08-30-19-00-34 +0000,nypost,Graphic video shows Israeli father throw himse...,https://nypost.com/2025/08/30/world-news/video...
175,39,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...
29,35,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...
104,32,2025-08-30-10-23-03 +0000,nypost,Former Ukrainian parliamentary speaker Andriy ...,https://nypost.com/2025/08/30/world-news/forme...
160,31,2025-08-30-01-00-00 +0000,wsj,Minnesota Gov. Tim Walz campaigned on the nati...,https://www.wsj.com/politics/elections/we-aske...
16,29,2025-08-30-19-54-53 +0000,nypost,Twisted T-shirts pay tribute to Minneapolis ch...,https://nypost.com/2025/08/30/us-news/website-...
136,28,2025-08-30-07-22-55 +0000,bbc,US blocks Palestinian leader from attending UN...,https://www.bbc.com/news/articles/cjdym32z9v7o...
62,27,2025-08-30-15-37-54 +0000,bbc,"Russia launches massive attack on Ukraine, as ...",https://www.bbc.com/news/articles/c4g614x9kqko...
116,25,2025-08-30-10-00-00 +0000,latimes,"California gas prices jump, bringing extra pai...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
